In [ ]:
from pytorch_model import load_wpod
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms, utils, models
from torch import nn
import torch
from src.label import Label, Shape
from src.utils import getWH, nms, im2single, IOU_centre_and_dims
from src.projection_utils import getRectPts, find_T_matrix
import time
from utils import *
from tqdm import tqdm
from torchviz import make_dot

In [ ]:
import sys
import cv2
import numpy as np
import traceback

import darknet.python.darknet as dn

from os.path 				import splitext, basename
from glob					import glob
from darknet.python.darknet import detect
from src.label				import dknet_label_conversion
from src.utils 				import nms

In [ ]:
pil_to_tensor = transforms.ToTensor()
tensor_to_pil = transforms.ToPILImage()

In [ ]:
class DLabel (Label):

    def __init__(self,cl,pts,prob):
        self.pts = pts
        tl = np.amin(pts,1)
        br = np.amax(pts,1)
        Label.__init__(self,cl,tl,br,prob)


def reconstruct(Iorig,I,Y,out_size,threshold=.9):

    net_stride 	= 2**4
    side = ((208. + 40.)/2.)/net_stride # 7.75
    
    Probs = Y[...,0]
    Affines = Y[...,2:]
    rx,ry = Y.shape[:2]
    ywh = Y.shape[1::-1]
    iwh = np.array(I.shape[1::-1],dtype=float).reshape((2,1))
    xx,yy = np.where(Probs>threshold)
    
    WH = getWH(I.shape)
    MN = WH/net_stride

    vxx = vyy = 0.5 #alpha

    base = lambda vx,vy: np.matrix([[-vx,-vy,1.],[vx,-vy,1.],[vx,vy,1.],[-vx,vy,1.]]).T
    labels = []

    for i in range(len(xx)):
        y,x = xx[i],yy[i]
        affine = Affines[y,x]
        prob = Probs[y,x]
        mn = np.array([float(x) + .5,float(y) + .5])
        A = np.reshape(affine,(2,3))
        A[0,0] = max(A[0,0],0.)
        A[1,1] = max(A[1,1],0.)
        pts = np.array(A*base(vxx,vyy)) #*alpha
        pts_MN_center_mn = pts*side
        pts_MN = pts_MN_center_mn + mn.reshape((2,1))

        pts_prop = pts_MN/MN.reshape((2,1))

        labels.append(DLabel(0,pts_prop,prob))

    final_labels = nms(labels,.1)
    TLps = []

    if len(final_labels):
        final_labels.sort(key=lambda x: x.prob(), reverse=True)
        for i,label in enumerate(final_labels):

            t_ptsh 	= getRectPts(0,0,out_size[0],out_size[1])
            ptsh 	= np.concatenate((label.pts*getWH(Iorig.shape).reshape((2,1)),np.ones((1,4))))
            H 		= find_T_matrix(ptsh,t_ptsh)
            Ilp 	= cv2.warpPerspective(Iorig,H,out_size,borderValue=.0)

            TLps.append(Ilp)

    return final_labels,TLps
    

def detect_lp(model,I,max_dim,net_step,out_size,threshold,masked_pattern=None,train=None):

    min_dim_img = min(I.shape[:2])
    factor 		= float(max_dim)/min_dim_img

    w,h = (np.array(I.shape[1::-1],dtype=float)*factor).astype(int).tolist()
    w += (w%net_step!=0)*(net_step - w%net_step)
    h += (h%net_step!=0)*(net_step - h%net_step)
    Iresized = cv2.resize(I,(w,h))
    print(Iresized.shape)
    Tn = Iresized.copy()
    Tn = Tn.reshape((1,Tn.shape[0],Tn.shape[1],Tn.shape[2]))
    Tn = torch.tensor(Tn, device=device).permute(0,3,1,2)
    #display(tensor_to_pil(Tn.cpu()[0]))
    start = time.time()
    model.eval()
    if masked_pattern is not None:
        Tn = masked_pattern*Tn
    if train:
        Yr = model(Tn).permute(0,2,3,1)
        return None,None,None,Yr,Tn
    else:
        display(tensor_to_pil(Tn.cpu()[0]))
        Yr = model(Tn).permute(0,2,3,1)
        Y2 = np.squeeze(Yr)
        Y2 = Y2.cpu().detach().numpy()
    elapsed = time.time() - start
    
    L,TLps = reconstruct(I,Iresized,Y2,out_size,threshold)

    return L,TLps,elapsed,Yr.cpu(),Tn
    
def getResizedImage(I,max_dim,net_step):
    min_dim_img = min(I.shape[:2])
    factor 		= float(max_dim)/min_dim_img

    w,h = (np.array(I.shape[1::-1],dtype=float)*factor).astype(int).tolist()
    w += (w%net_step!=0)*(net_step - w%net_step)
    h += (h%net_step!=0)*(net_step - h%net_step)
    Iresized = cv2.resize(I,(w,h))
    return Iresized

def augment_sample(I,pts,dim):

	maxsum,maxangle = 120,np.array([80.,80.,45.])
	angles = np.random.rand(3)*maxangle
	if angles.sum() > maxsum:
		angles = (angles/angles.sum())*(maxangle/maxangle.sum())

	I = im2single(I)
	iwh = getWH(I.shape)

	whratio = random.uniform(2.,4.)
	wsiz = random.uniform(dim*.2,dim*1.)
	
	hsiz = wsiz/whratio

	dx = random.uniform(0.,dim - wsiz)
	dy = random.uniform(0.,dim - hsiz)

	pph = getRectPts(dx,dy,dx+wsiz,dy+hsiz)
	pts = pts*iwh.reshape((2,1))
	T = find_T_matrix(pts2ptsh(pts),pph)

	H = perspective_transform((dim,dim),angles=angles)
	H = np.matmul(H,T)

	Iroi,pts = project(I,H,pts,dim)
	
	hsv_mod = np.random.rand(3).astype('float32')
	hsv_mod = (hsv_mod - .5)*.3
	hsv_mod[0] *= 360
	Iroi = hsv_transform(Iroi,hsv_mod)
	Iroi = np.clip(Iroi,0.,1.)

	pts = np.array(pts)

	if random.random() > .5:
		Iroi,pts = flip_image_and_pts(Iroi,pts)

	tl,br = pts.min(1),pts.max(1)
	llp = Label(0,tl,br)

	return Iroi,llp,pts

def l1(true,pred,szs):
	b,h,w,ch = szs
	res = torch.reshape(true-pred,(b,h*w*ch))
	res = torch.abs(res)
	res = torch.sum(res,1)
	return res

def logloss(Ptrue,Pred,szs,eps=10e-10):
	b,h,w,ch = szs
	Pred = torch.clamp(Pred,eps,1.)
	Pred = -torch.log(Pred)
	Pred = Pred*Ptrue
	Pred = torch.reshape(Pred,(b,h*w*ch))
	Pred = torch.sum(Pred,1)
	return Pred

def total_loss(Ytrue, Ypred):

    b = Ytrue.size()[0]
    h = Ytrue.size()[1]
    w = Ytrue.size()[2]

    obj_probs_true = Ytrue[...,0]
    obj_probs_pred = Ypred[...,0]

    #print((obj_probs_true>0).nonzero())
    #print((obj_probs_pred>0.5).nonzero())
    #print(obj_probs_true[obj_probs_true>0],obj_probs_pred[obj_probs_true>0])
    #print(obj_probs_true[obj_probs_pred>0.5],obj_probs_pred[obj_probs_pred>0.5])
    
    non_obj_probs_true = 1. - Ytrue[...,0]
    non_obj_probs_pred = Ypred[...,1]

    affine_pred	= Ypred[...,2:]
    pts_true 	= Ytrue[...,1:]

    affine_pred[...,0][affine_pred[...,0]<0] = 0
    affine_pred[...,4][affine_pred[...,4]<0] = 0
    affinex = torch.stack([affine_pred[...,0],affine_pred[...,1],affine_pred[...,2]],3)
    affiney = torch.stack([affine_pred[...,3],affine_pred[...,4],affine_pred[...,5]],3)
    v = 0.5
    base = torch.tensor([[[[-v,-v,1., v,-v,1., v,v,1., -v,v,1.]]]],device=device)
    base = base.repeat(b,h,w,1)
    pts = torch.zeros((b,h,w,0),device=device)

    for i in range(0,12,3):
        row = base[...,i:(i+3)]
        ptsx = torch.sum(affinex*row,3)
        ptsy = torch.sum(affiney*row,3)

        pts_xy = torch.stack([ptsx,ptsy],3)
        pts = (torch.cat([pts,pts_xy],3))

    flags = torch.reshape(obj_probs_true,(b,h,w,1))
    res   = 1.*l1(pts_true*flags,pts*flags,(b,h,w,4*2))
    res  += 1.*logloss(obj_probs_true,obj_probs_pred,(b,h,w,1))
    res  += 1.*logloss(non_obj_probs_true,non_obj_probs_pred,(b,h,w,1))
    return res

In [ ]:
def labels2output_map(label,lppts,dim,stride):

    #side = ((float(dim) + 40.)/2.)/stride # 7.75 when dim = 208 and stride = 16
    
    side = 7.75
    
    outsize1, outsize2 = int(dim[0]/stride), int(dim[1]/stride)
    Y  = np.zeros((outsize1,outsize2,2*4+1),dtype='float32')
    MN = np.array([outsize1,outsize2])
    WH = np.array([dim[0],dim[1]],dtype=float)

    tlx,tly = np.floor(np.maximum(label.tl(),0.)*MN).astype(int).tolist()
    brx,bry = np.ceil (np.minimum(label.br(),1.)*MN).astype(int).tolist()

    for x in range(tlx,brx):
        for y in range(tly,bry):

            mn = np.array([float(x) + .5, float(y) + .5])
            iou = IOU_centre_and_dims(mn/MN,label.wh(),label.cc(),label.wh())

            if iou > .5:

                p_WH = lppts*WH.reshape((2,1))
                p_MN = p_WH/stride

                p_MN_center_mn = p_MN - mn.reshape((2,1))

                p_side = p_MN_center_mn/side

                Y[x,y,0] = 1.
                Y[x,y,1:] = p_side.T.flatten()

    return Y

In [ ]:
def adjust_pts(pts,lroi):
	return pts*lroi.wh().reshape((2,1)) + lroi.tl().reshape((2,1))


output_dir = "./temp"

lp_threshold = .5

wpod_net = load_wpod()
wpod_net.to(device)
wpod_net.eval()
print('Searching for license plates using WPOD-NET')

tvehicle = cv2.imread('Plate_examples/lp_white_new.jpg')
#tvehicle = cv2.resize(tvehicle,(208,208))
#Ivehicle = Ivehicle[450:630, 480:1170]
plt.imshow(tvehicle)
print(tvehicle.shape)
#Ivehicle = cv2.bitwise_and(Ivehicle, Ivehicle, mask=cv2.bitwise_not(get_mask(Ivehicle)))
ratio = float(max(tvehicle.shape[:2]))/min(tvehicle.shape[:2])
side  = int(ratio*288.)
bound_dim = min(side + (side%(2**4)),608)
print("\t\tBound dim: %d, ratio: %f" % (bound_dim,ratio)) 

Llp,LlpImgs,_,Y3,_ = detect_lp(wpod_net,im2single(tvehicle),bound_dim,2**4,(240,80),lp_threshold)

if len(LlpImgs):
    print('found')
    Ilp = LlpImgs[0]
    Ilp = cv2.cvtColor(Ilp, cv2.COLOR_BGR2GRAY)
    Ilp = cv2.cvtColor(Ilp, cv2.COLOR_GRAY2BGR)

    s = Shape(Llp[0].pts)
    cv2.imwrite('./temp/lp.jpg',255*Ilp)
    plt.imshow(Ilp)
print(Y3.shape)

In [ ]:
tl,br = s.pts.min(1),s.pts.max(1)
llp = Label(0,tl,br)
print(tl,br)
Y = labels2output_map(llp,s.pts,(512,912),16)
Y = torch.tensor(Y)
Y = Y.unsqueeze(0)

In [ ]:
Y3 = Y3.to(device)
Y = Y.to(device)
print(Y.size(), Y3.size())
total_loss(Y,Y3)

In [ ]:
def Tmn(Yr):
    Tmn = torch.ones([Yr.size()[0],Yr.size()[1],Yr.size()[2],8])
    Yr[...,2][Yr[...,2]<0]=0
    Yr[...,5][Yr[...,5]<0]=0
    Tmn[...,0]= Yr[...,2]*-0.5+Yr[...,3]*-0.5+Yr[...,6]
    Tmn[...,1]= Yr[...,4]*-0.5+Yr[...,5]*-0.5+Yr[...,7]
    Tmn[...,2]= Yr[...,2]*0.5+Yr[...,3]*-0.5+Yr[...,6]
    Tmn[...,3]= Yr[...,4]*0.5+Yr[...,5]*-0.5+Yr[...,7]
    Tmn[...,4]= Yr[...,2]*0.5+Yr[...,3]*0.5+Yr[...,6]
    Tmn[...,5]= Yr[...,4]*0.5+Yr[...,5]*0.5+Yr[...,7]
    Tmn[...,6]= Yr[...,2]*-0.5+Yr[...,3]*0.5+Yr[...,6]
    Tmn[...,7]= Yr[...,4]*-0.5+Yr[...,5]*0.5+Yr[...,7]
    return Tmn
def get_affine_loss(Y,Yr):
    Y = torch.tensor(Y)
    a = torch.sum(torch.abs(Tmn(Yr)-Y[...,1:]),axis=-1)
    L_affine = Y[...,0]*a
    return torch.sum(torch.sum(L_affine,dim=1),dim=1)
print(Y.shape, Y3.shape)
temp = Tmn(Y3)
print(temp[0][0][0][1], Y[7][7][0], Y[7][7][2])
get_affine_loss(Y,Y3)

In [ ]:
def get_mask(Ivehicle,h=None,w=None,d=None):
    if h:
        height,width,depth = h,w,d
    else: height,width,depth = Ivehicle.shape
    x = Ivehicle.copy()
    x = cv2.resize(x,(height,width))
    circle_img = np.zeros((height,width), np.uint8)
    #cv2.rectangle(circle_img,(86,107),(212,152),280,thickness=-1)
    #cv2.rectangle(circle_img,(86,107),(212,152),280,thickness=-1)
    print(circle_img.shape)
    #circle_img = circle_img.reshape((1,circle_img.shape[0],circle_img.shape[1]))
    #circle_img = torch.tensor(circle_img).permute(0,1,2)
    #cv2.rectangle(circle_img,(480,480),(1170,570),255,thickness=-1)
    plt.imshow(cv2.bitwise_and(x, x, mask=circle_img))
    return circle_img
get_mask(tvehicle,288,288,3)

In [ ]:
account_resize = True
repeat_size=1

#For resize post convolution
if account_resize:
    repeat_size = int(3024/3024)
    img = Ivehicle.resize((208,208))
    model_img_size = img.size[0]
    img_t = pil_to_tensor(img)
    img_t = img_t.to(device)
    def resize2d(img, size):
        return (F.adaptive_avg_pool2d(Variable(img), size)).data
    def upsample2d(img, size=224):
        upsample = nn.Upsample(size=size, mode='bilinear', align_corners=False)
        return upsample(torch.unsqueeze(img, 0))[0]
else:
    model_img_size = 224

In [ ]:
model_img_size = 512

In [ ]:
# typical exposure is in form 1/n s. Available: 15, 20, 25 30, 40, 50, 60, 80, 100, 125, 160, 200, 250
exposure = 1000
exp_micros = 1000000/exposure          # get exposure in microseconds
img_ratio = 2268 / model_img_size      # every row in model is img_ratio rows in original image
model_tr = 10 * img_ratio              # multiply real tr (10 micros) by img_ratio to find model tr
conv_size = exp_micros / model_tr      # divide exposure time by tr to find convolution size
conv_size = int(conv_size)             # Need closest integer approximation. Won't cause a significant difference
conv_size

In [ ]:
#Ivehicle = cv2.imread('Plate_examples/lptest.jpg')
r = cv2.imread('Plate_examples/lp_red.jpg')
g = cv2.imread('Plate_examples/lp_green.jpg')
b = cv2.imread('Plate_examples/lp_blue.jpg')
white = cv2.imread('Plate_examples/lp_white_new.jpg')
canvas = np.zeros([2268,4032,3],dtype=np.uint8)
canvas[:,:,0] = b[:,:,0]
canvas[:,:,1] = g[:,:,1]
canvas[:,:,2] = r[:,:,2]
Ivehicle = white
#Ivehicle = cv2.resize(Ivehicle,(208,208))
print(Ivehicle.shape)
plt.imshow(white)
#print(w)
#print(torch.tensor(r).shape)
#display(tensor_to_pil(torch.tensor(w).permute(2,0,1)))
#Ivehicle = cv2.cvtColor(Ivehicle, cv2.COLOR_BGR2RGB)
#Ivehicle = cv2.resize(Ivehicle,(208,208))

In [ ]:
sz = model_img_size + conv_size - 1 # 300   #Length of input signal
c = 0    #Ambient light ratio
c_limits = [0,0]
batch = 8
channels = 3
# change of variable term to optimise on
w = torch.rand([channels,sz,1], requires_grad=True, dtype=torch.float, device=device)

#Create the mask to only illuminate the object
#mask = torch.tensor(get_mask(Ivehicle,288,288,3), dtype=torch.float, device=device)
#mask = torch.tensor(get_mask(Ivehicle,288,288,3), device=device)
mask = torch.ones([3,512,912], device=device)
mask = mask / torch.max(mask)
#mask = get_mask(Ivehicle,288,288,3)

#Target and original class labels
#orig = torch.tensor([classidx], dtype=torch.long, device=device)

#Model parameters
lr = 1e-1
n_epochs = 1000
optimizer = optim.SGD([w], lr=lr, momentum=0.9, nesterov=True)
#optimizer = optim.Adam([w], lr=lr)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
labelf = torch.flatten(torch.tensor(Y[...,0], dtype=torch.float, device=device))
labelnf = torch.flatten(torch.tensor(1-Y[...,0], dtype=torch.float, device=device))

In [ ]:
n_epochs = 500

In [ ]:
def train_run(update_frequency, print_frequency, n_epochs, image):
    #Track the loss to target and original class
    loss_list = []

    #obj_dict = {}
    #w = -1*w
    #Optimisation loop. initially untargeted
    for epoch in tqdm(range(n_epochs)):

        if channels==1:
            n_w = w.repeat(3,1,1)
        else:
            n_w = w
            
        #with torch.no_grad():
        #    torch.clamp_(n_w, min=-0.3)
        # For resize post convolution
        #if account_resize:
        #    n_w = torch.repeat_interleave(n_w, repeats=repeat_size, dim=1)
        
        sig_height = model_img_size + conv_size - 1
        gy, new_w = fttogy(n_w, batch, None, c_limits, sig_height, conv_size, 0, shifting=True)

        #with torch.no_grad():
        #    torch.clamp_(gy, min=0.1)
        #pattern = 1-mask + torch.mul(gy,mask)
        #new_w.retain_grad()
        #gy.retain_grad()
        #with torch.autograd.set_detect_anomaly(True):
        #Llp,LlpImgs,x,Yr,T = detect_lp(wpod_net,im2single(Ivehicle),bound_dim,2**4,(240,80),lp_threshold,gy,train=True)
        Tn = gy*image
        Yr = wpod_net(Tn).permute(0,2,3,1)
        #Yr.retain_grad()
        #loss = torch.sum(torch.stack([Yr[i][...,0][np.where(Y[...,0]>0.5)] for i in range(Yr.size()[0])]**2))
        
        loss = (torch.max(torch.max(Yr[:,:,:,0],dim=1)[0],dim=1)[0])**2
        loss = torch.sum(loss[loss>0.25])
        
        loss = -1*torch.sum(total_loss(Y,Yr))
        #print(make_dot(new_w))
        #loss.retain_grad()
        if epoch%print_frequency==0:
            #print(Yr[...,0][np.where(Y[...,0]>0.5)])
            loss_list.append(loss)
            
        loss.backward()

        if epoch%max(100,update_frequency)==0:
            #print(Yr[...,0][np.where(Y[...,0]>0.5)])
            #print(w.grad[0][:5], w.grad[1][:5], w.grad[2][:5])
            #print(w[0][:5],w[1][:5],w[2][:5])
            print(loss) 
            print(total_loss(Y,Yr))
            print(torch.max(torch.max(Yr[:,:,:,0],dim=1)[0],dim=1)[0])

        if epoch%update_frequency==0:
            #print('loss_grad',loss.grad)
            #print('gy_grad', gy.grad[gy.grad>0])
            #print('w_grad', w.grad)
            #print('new_w', new_w.grad[new_w.grad>0])
            #print(w.grad[0][:5])#, w.grad[1][:5], w.grad[2][:5])
            #print(w[0][:5],w[1][:5],w[2][:5])
            optimizer.step()
            #with torch.no_grad():
            #    torch.clamp_(w, min=-1)
            optimizer.zero_grad()
            del loss
            torch.cuda.empty_cache()
        #if epoch!=n_epochs-1:
        #    del inp
        #    del new_w
        #else:
            #saving w to be used for prediction
            #torch.save(n_w,'w_0.5_764.pt')

        #Code to check gpu allocation    
        '''
        for obj in gc.get_objects():
            try:
                if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                    #print(type(obj), obj.size())
                    if type(obj) not in obj_dict:
                        obj_dict[type(obj)] = 1
                    else:
                        obj_dict[type(obj)] += 1
            except: pass
        print(obj_dict)
        obj_dict.clear()
        '''
    return loss_list, Yr, gy, new_w, Tn


In [ ]:
Iresized = getResizedImage(im2single(Ivehicle),bound_dim,2**4)
Tn = Iresized.copy()
Tn = Tn.reshape((1,Tn.shape[0],Tn.shape[1],Tn.shape[2]))
Tn = torch.tensor(Tn, device=device).permute(0,3,1,2)
#display(tensor_to_pil(Tn.cpu()[0]))
print(Y.size())
if Y.size()[0]==1:
    Y = Y.repeat(8,1,1,1)
Y = Y.to(device)
print(Y.size())

In [ ]:
#View original loss and target loss
update_freqs = [10]
result = []
for up in update_freqs:
    '''
    t = np.full((channels,500,1),-0)
    for c in range(3):
        for ind in range(500):
            t[c][ind][0] = ((c+int(ind/10))%10)-5
    w = torch.tensor(t, requires_grad=True, dtype=torch.float, device=device)
    '''
    #w = torch.rand([channels,100,1], requires_grad=True, dtype=torch.float, device=device)
    #optimizer = optim.SGD([w], lr=100, momentum=0.9, nesterov=True)
    optimizer = optim.Adam([w], lr=0.5)
    result.append(train_run(up, 10, 500, Tn))
    plt.plot([x.item() for x in result[-1][0]], label=str(up))
plt.legend()
plt.show()

In [ ]:
image = result[0][4][4].cpu().detach().numpy().transpose(1, 2, 0)
print(image.shape)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
for i in range(batch):
    print(i)
    #print(torch.mean(result[0][1][i][:,:,0][np.where(Y[:,:,0]>0.5)]).item())
    print(result[0][1][i][:,:,0][result[0][1][i][:,:,0]>0.5].cpu().detach().numpy())

In [ ]:
image = result[0][2][0].repeat(1,1,912).cpu().detach().numpy().transpose(1, 2, 0)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
#torch.save(result[0][3],'Plate_examples/size_500_rand_w_clamp_1.pt')
plt.plot(torch.flatten(result[0][3][0][0]).detach().cpu(), label="w_b")
plt.legend()
plt.show()
plt.plot(torch.flatten(result[0][3][0][1]).detach().cpu(), label="w_g")
plt.legend()
plt.show()
plt.plot(torch.flatten(result[0][3][0][2]).detach().cpu(), label="w_r")
plt.legend()
plt.show()

In [ ]:
Llp,LlpImgs,_,Y2,_ = detect_lp(wpod_net,im2single(Ivehicle),bound_dim,2**4,(240,80),lp_threshold,result[0][2][2:3])
print(Y2[...,0][Y2[...,0]>0.1])
if len(LlpImgs):
    print('found')
    Ilp = LlpImgs[0]
    Ilp = cv2.cvtColor(Ilp, cv2.COLOR_BGR2GRAY)
    Ilp = cv2.cvtColor(Ilp, cv2.COLOR_GRAY2BGR)
    #Ilp = cv2.cvtColor(Ilp, cv2.COLOR_BGR2RGB)
    s = Shape(Llp[0].pts)
    cv2.imwrite('./temp/lp.jpg',255*Ilp)
    plt.imshow(Ilp)

In [ ]:
print(Y2[...,0][np.where(Y2[...,0]>0.5)])
print(torch.max(torch.max(Y2[:,:,:,0],dim=1)[0],dim=1)[0])

In [ ]:
print(w[0][:5])

In [ ]:
def applyMask(new_w, batch_limits, mask, c_limits, sig_height, conv_size, precision_depth=2):
    
    lay = torch.nn.Conv1d(1,1,conv_size)

    #Manually setting the weights and bias so the  shutter acts as a box filter
    lay.weight.data = torch.full([1,1,conv_size,1], 1/conv_size, requires_grad=True, dtype=torch.float, device=device)
    lay.bias.data = torch.zeros(1, requires_grad=True, dtype=torch.float, device=device)
    
    sz = new_w.shape[1]
    batch = batch_limits[1]-batch_limits[0]
    #stack the signal to fit the input size
    new_w = stack(new_w,sig_height)             
    
    # EOT sampling for ambient light and shift
    c = torch.rand([batch,1,1,1], device=device) * (c_limits[1] - c_limits[0]) + c_limits[0]
    shift = torch.tensor(range(batch_limits[0],batch_limits[1]), dtype=torch.int)
    #shift = torch.from_numpy(np.array(range(0,batch,1)))
    
    #Shift the signal
    new_w = shift_operation(new_w.unsqueeze(0).repeat(batch,1,1,1).view(-1, sig_height, 1), shift).view(batch,3,sig_height,1)
    
    #Fit w into the range [0,1]. new_w is the same as ft
    #new_w = .5 * (torch.tanh(ootn) + 1)
    
    #precision limit
    new_w = diff_round(new_w, precision_depth)
    
    #Convolution of ft and the shutter
    #gy = lay(new_w.unsqueeze(0).view([3,1,228,batch])).view([batch,3,224,1])
    gy = lay(new_w.transpose(0,3).transpose(0,1)).transpose(0,1).transpose(0,3)

    #Mask the signal to only affect the object
    if mask:
        gy_mask = torch.mul(gy,torch.transpose(mask,1,0))
    else:
        gy_mask = gy

    return (c + (1-c)*gy_mask), c

In [ ]:
def get_ocr(count):
    ocr_threshold = .4

    ocr_weights = bytes('data/ocr/ocr-net.weights', encoding="utf-8")
    ocr_netcfg  = bytes('data/ocr/ocr-net.cfg', encoding="utf-8")
    ocr_dataset = bytes('data/ocr/ocr-net.data', encoding="utf-8")

    ocr_net  = dn.load_net(ocr_netcfg, ocr_weights, 0)
    ocr_meta = dn.load_meta(ocr_dataset)

    imgs_path = bytes('./temp/lp'+str(count)+'.jpg', encoding="utf-8")

    bname = basename(splitext(img_path)[0])

    R = detect(ocr_net, ocr_meta, imgs_path ,thresh=ocr_threshold, nms=None)

    if len(R):

        L = dknet_label_conversion(R,240,80)
        L = nms(L,.45)

        L.sort(key=lambda x: x.tl()[0])
        lp_str = ''.join([chr(l.cl()) for l in L])

        return lp_str
    return ""

In [ ]:
ww = result[0][3]#torch.load('Plate_examples/size_100_init_striped_input_white_clamp_0.pt', map_location=torch.device('cpu'))
#w = torch.ones(w.size())
ww = ww.to(device)
ww = ww[0]
sig_height = model_img_size + conv_size - 1
max_accuracies = []
recovered_characters = []
for i in range(0,100,5): 
    gy, c = applyMask(ww, [i,i+5], None, [0,0], sig_height, conv_size, 0)
    Yr = wpod_net(gy*Tn).permute(0,2,3,1)
    max_accuracies += list(torch.max(torch.max(Yr[:,:,:,0],dim=1)[0],dim=1)[0].cpu().detach().numpy())
    for i in range(Yr.size()[0]):
        L,TLps = reconstruct(Ivehicle,getResizedImage(im2single(Ivehicle),bound_dim,2**4),Yr[i].cpu().detach().numpy(),(240,80),0.5)
        if len(TLps):
            Ilp = TLps[0]
            Ilp = cv2.cvtColor(Ilp, cv2.COLOR_BGR2GRAY)
            Ilp = cv2.cvtColor(Ilp, cv2.COLOR_GRAY2BGR)
            cv2.imwrite('./temp/lp'+str(i)+'.jpg',255*Ilp)
            ocr_text = get_ocr(i)
            recovered_characters.append(len([t for t in "BGJ6974" if t in ocr_text]))
    del gy
    del Yr
    torch.cuda.empty_cache()
#image = gy[0].repeat(1,1,912).cpu().detach().numpy().transpose(1, 2, 0)
#print(max_accuracies)
plt.plot(max_accuracies)
plt.show()
plt.plot(recovered_characters)
plt.show()
'''
image = (gy[0]*Tn)[0].cpu().detach().numpy().transpose(1, 2, 0)
print(image.shape)
b = image.copy()
b[:,:,1] = 0
b[:,:,2] = 0
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
'''